In [1]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine


In [2]:
filepath = "credits.csv"


In [3]:
# Create a dataframe from csv
df = pd.read_csv(filepath, delimiter=',')
# User list comprehension to create a list of lists from Dataframe rows
list_of_rows = [list(row) for row in df.values]
print(type(list_of_rows))


<class 'list'>


In [4]:
cast_dataframe = []
for row in list_of_rows:
    cast_list = eval(row[0])
    cast_record_df = pd.DataFrame(cast_list)
    cast_record_df['movie_id_number'] = row[2]
    cast_dataframe.append(cast_record_df)

cast_table = pd.concat(cast_dataframe, sort=False)
cast_table.reset_index(inplace=True, drop=True)
cast_table.head(5)

,cast_id,character,credit_id,gender,id,name,order,profile_path,movie_id_number
0,14.0,Woody (voice),52fe4284c3a36847f8024f95,2.0,31.0,Tom Hanks,0.0,/pQFoyx7rp09CJTAb932F2g8Nlho.jpg,862
1,15.0,Buzz Lightyear (voice),52fe4284c3a36847f8024f99,2.0,12898.0,Tim Allen,1.0,/uX2xVf6pMmPepxnvFWyBtjexzgY.jpg,862
2,16.0,Mr. Potato Head (voice),52fe4284c3a36847f8024f9d,2.0,7167.0,Don Rickles,2.0,/h5BcaDMPRVLHLDzbQavec4xfSdt.jpg,862
3,17.0,Slinky Dog (voice),52fe4284c3a36847f8024fa1,2.0,12899.0,Jim Varney,3.0,/eIo2jVVXYgjDtaHoF19Ll9vtW7h.jpg,862
4,18.0,Rex (voice),52fe4284c3a36847f8024fa5,2.0,12900.0,Wallace Shawn,4.0,/oGE6JqPP2xH4tNORKNqxbNPYi7u.jpg,862


In [5]:
crew_dataframe = []
for row in list_of_rows:
    crew_list = eval(row[1])
    crew_record_df = pd.DataFrame(crew_list)
    crew_record_df['movie_id_number'] = row[2]
    crew_dataframe.append(crew_record_df)

crew_table = pd.concat(crew_dataframe, sort=False)
crew_table.reset_index(inplace=True, drop=True)
crew_table.head(5)

,credit_id,department,gender,id,job,name,profile_path,movie_id_number
0,52fe4284c3a36847f8024f49,Directing,2.0,7879.0,Director,John Lasseter,/7EdqiNbr4FRjIhKHyPPdFfEEEFG.jpg,862
1,52fe4284c3a36847f8024f4f,Writing,2.0,12891.0,Screenplay,Joss Whedon,/dTiVsuaTVTeGmvkhcyJvKp2A5kr.jpg,862
2,52fe4284c3a36847f8024f55,Writing,2.0,7.0,Screenplay,Andrew Stanton,/pvQWsu0qc8JFQhMVJkTHuexUAa1.jpg,862
3,52fe4284c3a36847f8024f5b,Writing,2.0,12892.0,Screenplay,Joel Cohen,/dAubAiZcvKFbboWlj7oXOkZnTSu.jpg,862
4,52fe4284c3a36847f8024f61,Writing,0.0,12893.0,Screenplay,Alec Sokolow,/v79vlRYi94BZUQnkkyznbGUZLjT.jpg,862


In [6]:
cast_junction_table = cast_table[['movie_id_number','character','order','id']]
cast_junction_table = cast_junction_table.rename(columns={'index':'rec_key','id':'actor_id', 'movie_id_number':'movie_id'})
cast_junction_table.reset_index()

,index,movie_id,character,order,actor_id
0,0,862,Woody (voice),0.0,31.0
1,1,862,Buzz Lightyear (voice),1.0,12898.0
2,2,862,Mr. Potato Head (voice),2.0,7167.0
3,3,862,Slinky Dog (voice),3.0,12899.0
4,4,862,Rex (voice),4.0,12900.0
...,...,...,...,...,...
562469,562469,227506,,0.0,544742.0
562470,562470,227506,,1.0,1090923.0
562471,562471,227506,,2.0,1136422.0
562472,562472,227506,,3.0,1261758.0


In [7]:
crew_junction_table = crew_table[['movie_id_number','department','job','id']]
crew_junction_table = crew_junction_table.rename(columns={'index':'rec_key','id':'crew_id', 'movie_id_number':'movie_id'})
crew_junction_table.reset_index()

,index,movie_id,department,job,crew_id
0,0,862,Directing,Director,7879.0
1,1,862,Writing,Screenplay,12891.0
2,2,862,Writing,Screenplay,7.0
3,3,862,Writing,Screenplay,12892.0
4,4,862,Writing,Screenplay,12893.0
...,...,...,...,...,...
464309,464309,67758,Sound,Original Music Composer,549356.0
464310,464310,67758,Camera,Director of Photography,58818.0
464311,464311,227506,Directing,Director,1085341.0
464312,464312,227506,Production,Producer,1195656.0


In [8]:
rds_connection_string = "postgres:postgres@localhost:5432/etl_db"
engine = create_engine(f'postgresql://{rds_connection_string}')

In [9]:
engine.table_names()

['movies_data', 'actors', 'crew', 'crew_movie_junction', 'cast_movie_junction']

In [10]:
cast_junction_table.to_sql(name='cast_movie_junction', con=engine, if_exists='append', index=False)

In [11]:
crew_junction_table.to_sql(name='crew_movie_junction', con=engine, if_exists='append', index=False)

In [18]:
cast_table = cast_table[['id','name','gender']]
cast_table = cast_table.rename(columns={'id':'actor_id'})
cast_table = cast_table.drop_duplicates(subset=['actor_id'])
cast_table.reset_index()


,index,actor_id,name,gender
0,0,31.0,Tom Hanks,2.0
1,1,12898.0,Tim Allen,2.0
2,2,7167.0,Don Rickles,2.0
3,3,12899.0,Jim Varney,2.0
4,4,12900.0,Wallace Shawn,2.0
...,...,...,...,...
206153,562461,169498.0,Jeremy Lelliott,2.0
206154,562463,1177500.0,Jason Widener,0.0
206155,562465,93930.0,Kiko Ellsworth,2.0
206156,562468,1686379.0,Darrell Dubovsky,0.0


In [19]:
cast_table.to_sql(name='actors', con=engine, if_exists='append', index=False)

In [20]:
crew_table = crew_table[['id','name','gender']]
crew_table = crew_table.rename(columns={'id':'crew_id'})
crew_table = crew_table.drop_duplicates(subset=['crew_id'])
crew_table.reset_index()

,index,crew_id,name,gender
0,0,7879.0,John Lasseter,2.0
1,1,12891.0,Joss Whedon,2.0
2,2,7.0,Andrew Stanton,2.0
3,3,12892.0,Joel Cohen,2.0
4,4,12893.0,Alec Sokolow,0.0
...,...,...,...,...
161570,464298,1749862.0,Sahand Torabi,0.0
161571,464299,1749863.0,Homayoun Shajarian,0.0
161572,464302,1728582.0,Dante Perez,0.0
161573,464308,549355.0,Jeffrey Goldenberg,2.0


In [21]:
crew_table.to_sql(name='crew', con=engine, if_exists='append', index=False)